In [ ]:
# Ch09-3 Geometric Operations

In [4]:
# Libraries
import numpy as np

In [5]:
# Retrieve data
from Bio import PDB 
repository = PDB.PDBList() 
parser = PDB.PDBParser() 
repository.retrieve_pdb_file('1TUP', pdir='.', file_format='pdb') 
p53_1tup = parser.get_structure('P 53', 'pdb1tup.ent') 

Structure exists: './pdb1tup.ent' 


/Users/shanebrubaker/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6146.
  warnings.warn(
/Users/shanebrubaker/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 6147.
  warnings.warn(
/Users/shanebrubaker/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6148.
  warnings.warn(
/Users/shanebrubaker/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 6149.
  warnings.warn(
/Users/shanebrubaker/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain F is discontinuous at line 6171.
  warnings.warn(
/Users/shanebrubaker/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:10

In [6]:
# Recall residue types
my_residues = set() 
for residue in p53_1tup.get_residues(): 
    my_residues.add(residue.id[0]) 
print(my_residues) 

{'H_ZN', 'W', ' '}


In [7]:
# Compute masses for chains, zincs, and waters

import numpy as np

# Function to compute mass
def get_mass(atoms, accept_fun=lambda x: True):
    return sum([atom.mass for atom in atoms if accept_fun(atom)])

# Extract chain names
chain_names = [chain.id for chain in p53_1tup.get_chains()]

# Initialize NumPy array for masses
my_mass = np.ndarray((len(chain_names), 3))

# Iterate over chains to compute mass
for i, chain in enumerate(p53_1tup.get_chains()):
    my_mass[i, 0] = get_mass(chain.get_atoms())

print("Mass array:", my_mass)


Mass array: [[6.42003092e+03 0.00000000e+00 4.47213595e-01]
 [6.48219602e+03 1.82574186e-01 4.47213595e-01]
 [2.37161442e+04 3.65148372e-01 4.47213595e-01]
 [2.14881464e+04 5.47722558e-01 4.47213595e-01]
 [2.17461774e+04 7.30296743e-01 4.47213595e-01]]


In [8]:
# Compute geometric center and center of mass
def get_center(atoms, 
    weight_fun=lambda atom: 1 if atom.parent.id[0] != 'W' else 0): 
    xsum = ysum = zsum = 0.0 
    acum = 0.0 
    for atom in atoms: 
        x, y, z = atom.coord 
        weight = weight_fun(atom) 
        acum += weight 
        xsum += weight * x 
        ysum += weight * y 
        zsum += weight * z 
    return xsum / acum, ysum / acum, zsum / acum 
print(get_center(p53_1tup.get_atoms())) 
print(get_center(p53_1tup.get_atoms(), 
    weight_fun=lambda atom: atom.mass if atom.parent.id[0] != 'W' else 0)) 

(58.296333984624454, 15.48705585839803, 78.40529502795778)
(58.12261138092647, 15.751286500376127, 78.37537036885965)


In [10]:
# Compute center of mass and geometric center of each chain
import pandas as pd
my_center = np.ndarray((len(chain_names), 6)) 
for i, chain in enumerate(p53_1tup.get_chains()): 
    x, y, z = get_center(chain.get_atoms()) 
    my_center[i, 0] = x 
    my_center[i, 1] = y 
    my_center[i, 2] = z 
    x, y, z = get_center(chain.get_atoms(), 
        weight_fun=lambda atom: atom.mass if atom.parent.id[0] != 'W' else 0) 
    my_center[i, 3] = x 
    my_center[i, 4] = y 
    my_center[i, 5] = z 
weights = pd.DataFrame(my_center, index=chain_names, 
    columns=['X', 'Y', 'Z', 'X (Mass)', 'Y (Mass)', 'Z (Mass)']) 
print(weights) 

           X          Y          Z   X (Mass)   Y (Mass)   Z (Mass)
E  49.727231  32.744879  81.253417  49.708513  32.759725  81.207395
F  51.982368  33.843370  81.578795  52.002223  33.820064  81.624394
A  72.990763  28.825429  56.714012  72.822668  28.810327  56.716117
B  67.810026  12.624435  88.656590  67.729100  12.724130  88.545659
C  38.221565  -5.010494  88.293141  38.169364  -4.915395  88.166711
